In [1]:
import numpy as np
import pandas as pd
import re

import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/Consolidated_purchases/OK_consol_purchases_march1_april21.csv")
consolidated_df.head()

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
0,252033,M123033004,-172.0,69025,-59.0,-123.9,-123.9,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
1,252099,0252099,153.3,69025,15.0,31.5,31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
2,252100,0252099,-153.3,69025,-15.0,-31.5,-31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
3,252099,0252099,153.3,68933,14.0,29.4,29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0
4,252100,0252099,-153.3,68933,-14.0,-29.4,-29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0


In [3]:
categories_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/ghge_factors.csv")
categories_df


,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316
...,...,...,...
58,59,manually adjusted,0.0000
59,60,human labor,0.0000
60,61,kitchen supplies,0.0000
61,62,mushrooms,1.5000


In [4]:
items_list = pd.read_csv('C:/Users/smvan/CFFS-S23/CFFS-22-23/data/mapping/Items_List_Assigned.csv')
items_list

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT
...,...,...,...,...,...,...,...,...
2072,I-8305,6.0,CHEESE GOAT LOG,1.00,Kg,1.0,Kg,DAIRY
2073,I-4776,38.0,PARSNIPS (20 LBS) BC,1.00,lb,1.0,lb,PRODUCE
2074,I-9274,41.0,PASTA GNOCCHI POTATO,6.00,Kg,1.0,Kg,FOOD - GROCERY
2075,I-11093,51.0,POTATO RED BABY B'S,50.00,lb,1.0,lb,PRODUCE


In [5]:
consolidated_df.columns

Index(['invoice_counter', 'invoice_num', 'invoice_total', 'item_num', 'qty',
       'line_amount', 'adj_line_amount', 'uom', 'unit_cost', 'item_descrip',
       'invoice_date', 'supplier_name', 'store_num', 'type_bill_credit',
       'cost_over_ride', 'line_amount_qty', 'case_uom', 'supplier_num',
       'prod_group_num', 'prod_group_descrip', 'order_code', 'compute_0022',
       'pak_qty', 'case_qty', 'purchase_uom', 'cat_num', 'cat_descrip',
       'has_credit', 'credit_pak_amount'],
      dtype='object')

In [6]:
# if item_num == items_list ItemId then match the category with it
for index, row in consolidated_df.iterrows():
    cos_id = "I-" + str(row['item_num'])
    consolidated_df.loc[index, 'item_num'] = cos_id
    print(cos_id)

I-69025
I-69025
I-69025
I-68933
I-68933
I-68932
I-68932
I-68934
I-68934
I-68931
I-68931
I-63023
I-63023
I-63024
I-63024
I-2937
I-2937
I-2937
I-2937
I-2937
I-2937
I-2937
I-2937
I-2937
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-62911
I-13791
I-13791
I-13791
I-13791
I-13791
I-13791
I-16095
I-16095
I-16095
I-16095
I-16095
I-16095
I-16095
I-16095
I-16095
I-16095
I-16095
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1020
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241
I-1241

I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-68718
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1197
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-1120
I-22490
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-9641
I-19634
I-19634
I-19634
I-19634
I-19634
I-19634
I-19634
I-1963

I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3337
I-3338
I-3340
I-3340
I-3341
I-3342
I-3342
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-60772
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3348
I-3352
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I-3355
I

I-4010
I-4010
I-4010
I-4010
I-4010
I-4010
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-4016
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-19665
I-8363
I-8363
I-8363
I-8363
I-8363
I-8363
I-8363
I-8363
I-8363
I-836

I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-4690
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-69452
I-4677
I-4677
I-4677
I-4677
I-69725
I-69725
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-4693
I-69726
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4844
I-4

I-62853
I-62853
I-62853
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62850
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849
I-62849


I-2422
I-2422
I-5490
I-5490
I-5490
I-15334
I-5500
I-5500
I-5500
I-5500
I-5500
I-5500
I-5500
I-2428
I-5512
I-5512
I-5512
I-5512
I-5512
I-5512
I-5512
I-5512
I-5512
I-5512
I-5512
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5516
I-5526
I-5526
I-5526
I-5526
I-5526
I-5526
I-5526
I-5526
I-57553
I-5527
I-5527
I-5527
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-5528
I-30507
I-30507
I-30507


In [7]:
consolidated_df.head(60)

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
0,252033,M123033004,-172.00,I-69025,-59.0,-123.90,-123.90,each,2.10,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
1,252099,0252099,153.30,I-69025,15.0,31.50,31.50,each,2.10,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
2,252100,0252099,-153.30,I-69025,-15.0,-31.50,-31.50,each,2.10,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
3,252099,0252099,153.30,I-68933,14.0,29.40,29.40,each,2.10,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0
4,252100,0252099,-153.30,I-68933,-14.0,-29.40,-29.40,each,2.10,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0
5,252099,0252099,153.30,I-68932,12.0,25.20,25.20,each,2.10,CANDIED COCONUT CASH,...,CANDIES SNACKS,KK052116,N,1.0,1.0,each,7,Candy,N,0
6,252100,0252099,-153.30,I-68932,-12.0,-25.20,-25.20,each,2.10,CANDIED COCONUT CASH,...,CANDIES SNACKS,KK052116,N,1.0,1.0,each,7,Candy,N,0
7,252099,0252099,153.30,I-68934,14.0,29.40,29.40,each,2.10,CANDIED IRISH CREAM,...,CANDIES SNACKS,KK051095,N,1.0,1.0,each,7,Candy,N,0
8,252100,0252099,-153.30,I-68934,-14.0,-29.40,-29.40,each,2.10,CANDIED IRISH CREAM,...,CANDIES SNACKS,KK051095,N,1.0,1.0,each,7,Candy,N,0
9,252099,0252099,153.30,I-68931,18.0,37.80,37.80,each,2.10,CANDIED SALT CARAMEL,...,CANDIES SNACKS,KK052117,N,1.0,1.0,each,7,Candy,N,0


In [8]:
# Create an empty list to store the matching CategoryID values
category_ids = []

# Iterate over each row in consolidated_df
for index, row in consolidated_df.iterrows():
    item_num = row['item_num']
    
    if item_num in items_list['ItemId'].values:
        category_id = items_list.loc[items_list['ItemId'] == item_num, 'CategoryID'].values[0]
        category_ids.append(category_id)
    else:
        category_ids.append(None)  # or any other value you prefer for non-matching rows

# Add the category_ids list as a new column to consolidated_df
consolidated_df['Items_assign_category_ID'] = category_ids

In [9]:
consolidated_df.head(60)

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Items_assign_category_ID
0,252033,M123033004,-172.00,I-69025,-59.0,-123.90,-123.90,each,2.10,CANDIED CINNAMON ALMOND,...,KK051094,N,1.0,1.0,each,7,Candy,N,0,NaN
1,252099,0252099,153.30,I-69025,15.0,31.50,31.50,each,2.10,CANDIED CINNAMON ALMOND,...,KK051094,N,1.0,1.0,each,7,Candy,N,0,NaN
2,252100,0252099,-153.30,I-69025,-15.0,-31.50,-31.50,each,2.10,CANDIED CINNAMON ALMOND,...,KK051094,N,1.0,1.0,each,7,Candy,N,0,NaN
3,252099,0252099,153.30,I-68933,14.0,29.40,29.40,each,2.10,CANDIED COCONUT ALMO,...,KK051093,N,1.0,1.0,each,7,Candy,N,0,NaN
4,252100,0252099,-153.30,I-68933,-14.0,-29.40,-29.40,each,2.10,CANDIED COCONUT ALMO,...,KK051093,N,1.0,1.0,each,7,Candy,N,0,NaN
5,252099,0252099,153.30,I-68932,12.0,25.20,25.20,each,2.10,CANDIED COCONUT CASH,...,KK052116,N,1.0,1.0,each,7,Candy,N,0,NaN
6,252100,0252099,-153.30,I-68932,-12.0,-25.20,-25.20,each,2.10,CANDIED COCONUT CASH,...,KK052116,N,1.0,1.0,each,7,Candy,N,0,NaN
7,252099,0252099,153.30,I-68934,14.0,29.40,29.40,each,2.10,CANDIED IRISH CREAM,...,KK051095,N,1.0,1.0,each,7,Candy,N,0,NaN
8,252100,0252099,-153.30,I-68934,-14.0,-29.40,-29.40,each,2.10,CANDIED IRISH CREAM,...,KK051095,N,1.0,1.0,each,7,Candy,N,0,NaN
9,252099,0252099,153.30,I-68931,18.0,37.80,37.80,each,2.10,CANDIED SALT CARAMEL,...,KK052117,N,1.0,1.0,each,7,Candy,N,0,NaN


In [10]:
# Filter rows with NaN values in "Items_assign_category_ID" and unique item_num
filtered_df = consolidated_df[~pd.isnull(consolidated_df['Items_assign_category_ID']) & ~consolidated_df['item_num'].duplicated(keep=False)]

# Print the filtered DataFrame
filtered_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Items_assign_category_ID
619,248536,71993398,837.60,I-3222,1.0,69.14,69.14,cs,69.14,CHOC CHIPPER SUPREME 4000CT,...,3030417,N,1.0,12.00,cs,2,Food,N,0,24.0
916,252153,72014855,354.90,I-1115,1.0,88.04,88.04,cs,88.04,DANISH MINI ASSORTED,...,GSDA220,N,1.0,120.00,cs,2,Food,N,0,24.0
1012,251391,9001425804,4111.73,I-2667,6.0,340.92,340.92,cs,56.82,PIE LEMON MERINGUE CKD10I,...,1043008,N,1.0,6.00,cs,2,Food,N,0,24.0
1545,250260,9001301576,4139.59,I-43559,3.0,261.96,261.96,cs,87.32,"PASTRY PUFF ROUND HIGH RISE 5""",...,5250717,N,1.0,200.00,CA,2,Food,N,0,24.0
2270,248598,9001155910,938.26,I-1695,6.0,192.06,192.06,cs,32.01,MONSTER ABSOLUTE,...,1239873,N,1.0,12.00,cs,2,Food,N,0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,250749,9001312371,7178.28,I-6870,1.0,19.84,19.84,cs,19.84,SALT SEA FINE AURORA,...,1251283,N,1.0,11.34,cs,2,Food,N,0,56.0
17518,250164,9001312375,717.13,I-3501,1.0,44.67,44.67,cs,44.67,SALT TABLE IODIZED,...,2160005,N,1.0,24.00,cs,2,Food,N,0,56.0
17526,249752,9001238074,2475.89,I-8038,1.0,221.04,221.04,ea,221.04,SEASONING CARIBBEAN JERK,...,181885,N,454.0,1.00,each,2,Food,Y,0,54.0
17530,252897,9001524702,3222.39,I-3312,2.0,22.16,22.16,cs,11.08,SEED FENNEL WHOLE,...,181850,N,16.0,1.00,pak,2,Food,Y,0,26.0


In [11]:
assigned_data = pd.read_csv('C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/GHG_assigned_data.csv')
assigned_data

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
0,252033,M123033004,-172.00,69025,-59.0,-123.90,-123.90,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
1,252099,0252099,153.30,69025,15.0,31.50,31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
2,252100,0252099,-153.30,69025,-15.0,-31.50,-31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
3,252099,0252099,153.30,68933,14.0,29.40,29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
4,252100,0252099,-153.30,68933,-14.0,-29.40,-29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18381,252537,9001537618,3899.87,5528,4.0,208.92,208.92,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0
18382,252458,9001537197,7606.70,5528,1.0,52.23,52.23,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0
18383,250022,9001206712,8283.19,30507,3.0,225.57,225.57,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0
18384,250792,9001333265,2352.89,30507,1.0,75.19,75.19,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,kitchen supplies,61.0


In [12]:
# filtered_df.rename(columns={'Items_assign_category_ID': 'Category ID'}, inplace=True)

for index, row in assigned_data.iterrows():
    cos_id = "I-" + str(row['item_num'])
    assigned_data.loc[index, 'item_num'] = cos_id

In [13]:
for index, row in filtered_df.iterrows():
    get_r = assigned_data.loc[assigned_data['item_num'] == row['item_num']]
    if not get_r.empty:
        Category_ID = get_r['Category ID'].values[0]
        filtered_df.at[index, 'Category ID'] = Category_ID

            

C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\3312780728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.at[index, 'Category ID'] = Category_ID


In [14]:
filtered_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Items_assign_category_ID,Category ID
619,248536,71993398,837.60,I-3222,1.0,69.14,69.14,cs,69.14,CHOC CHIPPER SUPREME 4000CT,...,N,1.0,12.00,cs,2,Food,N,0,24.0,24.0
916,252153,72014855,354.90,I-1115,1.0,88.04,88.04,cs,88.04,DANISH MINI ASSORTED,...,N,1.0,120.00,cs,2,Food,N,0,24.0,24.0
1012,251391,9001425804,4111.73,I-2667,6.0,340.92,340.92,cs,56.82,PIE LEMON MERINGUE CKD10I,...,N,1.0,6.00,cs,2,Food,N,0,24.0,24.0
1545,250260,9001301576,4139.59,I-43559,3.0,261.96,261.96,cs,87.32,"PASTRY PUFF ROUND HIGH RISE 5""",...,N,1.0,200.00,CA,2,Food,N,0,24.0,24.0
2270,248598,9001155910,938.26,I-1695,6.0,192.06,192.06,cs,32.01,MONSTER ABSOLUTE,...,N,1.0,12.00,cs,2,Food,N,0,55.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,250749,9001312371,7178.28,I-6870,1.0,19.84,19.84,cs,19.84,SALT SEA FINE AURORA,...,N,1.0,11.34,cs,2,Food,N,0,56.0,56.0
17518,250164,9001312375,717.13,I-3501,1.0,44.67,44.67,cs,44.67,SALT TABLE IODIZED,...,N,1.0,24.00,cs,2,Food,N,0,56.0,56.0
17526,249752,9001238074,2475.89,I-8038,1.0,221.04,221.04,ea,221.04,SEASONING CARIBBEAN JERK,...,N,454.0,1.00,each,2,Food,Y,0,54.0,54.0
17530,252897,9001524702,3222.39,I-3312,2.0,22.16,22.16,cs,11.08,SEED FENNEL WHOLE,...,N,16.0,1.00,pak,2,Food,Y,0,26.0,26.0


In [15]:
need_to_manually_check_data = pd.DataFrame()
for index, row in filtered_df.iterrows():
    if row['Items_assign_category_ID'] != row['Category ID']:
        need_to_manually_check_data = need_to_manually_check_data.append(row)


C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\2593731899.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  need_to_manually_check_data = need_to_manually_check_data.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\2593731899.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  need_to_manually_check_data = need_to_manually_check_data.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\2593731899.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  need_to_manually_check_data = need_to_manually_check_data.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\2593731899.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

In [16]:
# check to see if automated approach is correctly assigning items to the right category

need_to_manually_check_data

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Items_assign_category_ID,Category ID
5865,248795,10475,1592.30,I-55475,60.0,882.00,882.00,pak,14.70,BASE CHOCO FNZ YOGURT,...,N,1.600,1.0,pak,2,Food,N,0,58.0,52.0
5871,248795,10475,1592.30,I-55476,48.0,676.80,676.80,pak,14.10,BASE VANILLA FNZ YOGURT,...,N,1.600,1.0,pak,2,Food,N,0,58.0,10.0
6036,251531,9001425820,8341.31,I-63034,2.0,123.78,123.78,cs,61.89,BURGER VEG MALIBU GARDENBURGER,...,N,1.000,48.0,cs,2,Food,Y,0,59.0,40.0
6061,253157,7615002,302.01,I-17680,3.0,145.53,145.53,cs,48.51,CEREAL CORNPUFF GORILA MNCH GF,...,N,1.000,12.0,cs,2,Food,N,0,22.0,21.0
6140,248400,IN161410,413.36,I-37478,4.0,75.80,75.80,lb,18.95,CHILE ARBOL,...,N,1.000,1000.0,Kg,2,Food,N,0,40.0,54.0
6338,250251,IN162664,316.75,I-39593,72.0,303.12,303.12,can,4.21,JACKFRUIT,...,N,550.000,24.0,cs,2,Food,Y,0,35.0,31.0
6346,250637,9001312357,3604.58,I-3327,1.0,29.34,29.34,cs,29.34,JELLY GRAPE PURE PORT TFC,...,N,1.000,200.0,cs,2,Food,N,0,59.0,58.0
6348,248362,9001094414,7971.34,I-3333,4.0,63.24,63.24,cs,15.81,JELLY POWDER PEACH TFC,...,N,1.000,2.0,cs,2,Food,N,0,59.0,54.0
6349,248362,9001094414,7971.34,I-3334,4.0,63.24,63.24,cs,15.81,JELLY POWDER RASPB TFC,...,N,1.000,2.0,cs,2,Food,N,0,59.0,54.0
6350,248362,9001094414,7971.34,I-3335,6.0,94.86,94.86,cs,15.81,JELLY POWDER STRAWB TFC,...,N,1.000,2.0,cs,2,Food,N,0,59.0,54.0


In [17]:
# Unfilter rows with NaN values in "Items_assign_category_ID" and unique item_num
unfiltered_df = consolidated_df[pd.isnull(consolidated_df['Items_assign_category_ID']) & ~consolidated_df['item_num'].duplicated(keep=False)]

for index, row in unfiltered_df.iterrows():
    get_r = assigned_data.loc[assigned_data['item_num'] == row['item_num']]
    if not get_r.empty:
        Category_ID = get_r['Category ID'].values[0]
        unfiltered_df.at[index, 'Category ID'] = Category_ID

C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\4282006849.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unfiltered_df.at[index, 'Category ID'] = Category_ID


In [18]:
not_assigned_from_list_method = pd.DataFrame()
for index, row in unfiltered_df.iterrows():
    if row['Items_assign_category_ID'] != row['Category ID']:
        not_assigned_from_list_method = not_assigned_from_list_method.append(row)


C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  not_assigned_from_list_method = not_assigned_from_list_method.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  not_assigned_from_list_method = not_assigned_from_list_method.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  not_assigned_from_list_method = not_assigned_from_list_method.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  not_assigned_from_list_method = not_assigned_from_list_method.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  not_assigned_from_list_method = not_assigned_from_list_method.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  not_assigned_from_list_method = not_assigned_from_list_method.append(row)
C:\Users\smvan\AppData\Local\Temp\ipykernel_21860\1361021378.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

In [19]:
unfiltered_df.to_csv('C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/selected_unassigned_GHG_categories.csv', index = False)

In [23]:
unfiltered_df.shape

(76, 31)

In [20]:
unfiltered_df.head(60)

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Items_assign_category_ID,Category ID
308,249028,71995351,1364.86,I-53518,3.00,86.670,86.670,SHEET,28.89,BROWNIE TIGER,...,N,1.00,1.0,cs,2,Food,N,0,NaN,24.0
370,248562,365081,737.70,I-64920,2.00,84.000,84.000,DOZ,42.00,CAKE CARROT,...,N,1.00,12.0,DOZ,2,Food,N,0,NaN,24.0
928,249456,26207,226.44,I-1284,1.00,32.960,32.960,CA,32.96,DATE SQUARE GF,...,N,1.00,12.0,CA,2,Food,Y,0,NaN,24.0
1404,251903,9001459647,6270.89,I-1086,4.00,218.560,218.560,cs,54.64,COOKIE DOUGH CHOC CHIP N/HYDRO,...,N,1.00,128.0,cs,2,Food,Y,0,NaN,24.0
1523,250598,72004170,1143.20,I-40947,1.00,56.800,56.800,cs,56.80,LADY FINGERS 15/400GM,...,N,400.00,15.0,cs,2,Food,Y,0,NaN,24.0
1913,249752,9001238074,2475.89,I-64431,1.00,31.380,31.380,CA,31.38,CORE POWER STRAW 414ML,...,N,1.00,12.0,CA,2,Food,N,0,NaN,55.0
2062,248072,3192768230,740.23,I-61326,1.00,20.300,20.300,cs,20.30,ICED TEA PEACE STRAW LEMN LOVE,...,N,1.00,12.0,cs,2,Food,N,0,NaN,55.0
2522,248964,INV1215107,236.08,I-13756,2.00,25.420,25.420,750ML,12.71,SYRUP MONIN CARAMEL,...,N,0.75,1.0,750ML,2,Food,Y,0,NaN,43.0
2523,248964,INV1215107,236.08,I-6037,4.00,50.840,50.840,bottle,12.71,SYRUP MONIN HAZELNUT,...,N,0.75,1.0,bottle,2,Food,Y,0,NaN,43.0
2524,252207,INV1215570,1198.68,I-1577,6.00,76.260,76.260,750ML,12.71,SYRUP MONIN LEMON,...,N,1.00,1.0,750ML,2,Food,N,0,NaN,43.0


In [21]:
unfiltered_df.iloc[60:78, :]

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Items_assign_category_ID,Category ID
17650,249846,9001238094,6740.16,I-6896,2.0,307.90,307.90,cs,153.9500,CLEANER FLOOR MAXXX DUAL ACT,...,N,1.0,1.0,cs,4,Misc.,N,0,NaN,61.0
17655,248384,9001144506,4418.22,I-2016,1.0,220.74,220.74,JUG,220.7400,CLEANER FLOORWASH N WALK,...,N,1.0,1.0,JUG,4,Misc.,N,0,NaN,61.0
17656,248247,9001132138,511.57,I-5365,1.0,105.12,105.12,cs,105.1200,CLEANER GRILL GREASESTRIP PLUS,...,N,1.0,6.0,cs,4,Misc.,N,0,NaN,61.0
17666,251813,9001447441,836.20,I-51500,1.0,162.11,162.11,cs,162.1100,CLEANER TABLET CARE RINSE,...,N,1.0,150.0,cs,4,Misc.,Y,0,NaN,61.0
17669,252241,144196,431.81,I-29517,12.0,29.20,29.20,box,2.4333,CLEANING TABLET CAFIZA ESPRSS,...,N,1.0,100.0,box,4,Misc.,N,0,NaN,61.0
17806,249904,IN107888,390.42,I-2353,2.0,64.92,64.92,cs,32.4600,"9""X6"" 2-COMPARTMENT CLAMSHELL",...,N,1.0,200.0,cs,4,Misc.,N,0,NaN,61.0
17835,251781,650205,546.97,I-60374,2.0,244.69,244.69,cs,122.3450,BOX T/O #8 KRAFT 6.87X5.5X2.5,...,N,1.0,300.0,cs,4,Misc.,N,0,NaN,61.0
17852,249064,643636,577.85,I-5371,1.0,103.34,103.34,cs,103.3400,CONT ECO HOT FOOD 8Z PLA LINED,...,N,1.0,1000.0,cs,4,Misc.,N,0,NaN,61.0
17933,250208,9001271330,3251.50,I-5465,2.0,195.20,195.20,cs,97.6000,LINER BASKET BLK/WHT CHECK 12X,...,N,1.0,2000.0,cs,4,Misc.,N,0,NaN,61.0
18034,250308,9001283316,7626.41,I-20435,1.0,27.99,27.99,cs,27.9900,TISSUE FACIAL BOX WHT 2PLY 100,...,N,1.0,36.0,cs,4,Misc.,N,0,NaN,61.0


In [22]:
consolidated_df.to_csv('C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/GHG_categories_assigned_with_Items_List.csv', index = False)